In [2]:
import numpy as np
import tensorflow as tf
#import maxout
import highway_maxout as hmn
import utils
import dataset as ds

Using TensorFlow backend.


In [3]:
#======= FLAGS ==========
FLAGS = tf.app.flags.FLAGS
tf.app.flags.DEFINE_integer('maxout_layer_size', 8, 'Maxout layer size')
tf.app.flags.DEFINE_integer('maxout_pooling_size', 16, 'Maxout pooling size')
tf.app.flags.DEFINE_integer('lstm_size', 10, 'LSTM cell internal size')

In [4]:
# remove all variables
tf.reset_default_graph();

lstm_size = 10
#question_size = 5
#document_size = 8
word_vector_size = 300
maxout_pooling_size = 16
max_decoder_iterations = 4
maxout_layer_size = 8;
max_epoch = 1;
max_sequence_length = 11
#training_set_size = 100;

# 
question_ph = tf.placeholder(tf.float32, [1, max_sequence_length, word_vector_size])
document_ph = tf.placeholder(tf.float32, [1, max_sequence_length, word_vector_size])

# LSTM cell initialization
lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)


# LSTM cells for Bi-LSTM for COATINATION ENCODER
lstm_cenc_fw = tf.contrib.rnn.BasicLSTMCell(lstm_size)
lstm_cenc_bw = tf.contrib.rnn.BasicLSTMCell(lstm_size)

# create lstm cell for DYNAMIC POINTING DECODER
lstm_dec = tf.contrib.rnn.BasicLSTMCell(lstm_size)
# get lstm initial state of zeroes
lstm_dec_state = lstm_dec.zero_state(1, tf.float32)
start_pos = 0; # generate random between (0, document_size-1)
end_pos = 0;   # generate random between (0, document_size-1)

# create sentinel vector variable for both encodings 
#with tf.variable_scope("scope1") as scope:
sentinel_q = tf.get_variable("sentinel_q", [ lstm_size , 1], initializer = tf.random_normal_initializer())
sentinel_d = tf.get_variable("sentinel_d", [ lstm_size , 1], initializer = tf.random_normal_initializer()) 

# optimizer

optimizer = tf.train.AdamOptimizer()
    

In [5]:

# r = lstm(inputs = tf.convert_to_tensor([[1,2], [2,3]], dtype=tf.float32), state = zero_state_q)

def length(sequence):
  used = tf.sign(tf.reduce_max(tf.abs(sequence), 2))
  length = tf.reduce_sum(used, 1)
  length = tf.cast(length, tf.int32)
  return length

'''
transform tensor of shape [1, question_size, word_vector_size] to list of tensors of shape [1, word_vector_size]
of length question_size. first dimenstion is batch size = 1
'''

#print(tf.shape(question_ph)[1])
question_input = tf.unstack(question_ph, max_sequence_length, 1)
document_input = tf.unstack(document_ph, max_sequence_length, 1)
#print(x)

# we use the same LSTM for both encodings
outputs_q, state_q = tf.contrib.rnn.static_rnn(lstm, inputs = question_input, sequence_length = length(question_ph), dtype=tf.float32)
outputs_d, state_d = tf.contrib.rnn.static_rnn(lstm, inputs = document_input, sequence_length = length(document_ph), dtype=tf.float32)

# "squeeze" transforms list of tensors of shape [1, lstm_size] of length L to tensor of shape [L, lstm_size]
que_enc = tf.transpose(tf.squeeze(outputs_q))
doc_enc = tf.transpose(tf.squeeze(outputs_d))
document_size =tf.shape(doc_enc)[1]


# append sentinel vector for both encodings 
doc_enc_sentinel = tf.concat([doc_enc, sentinel_d], axis = 1)
que_enc_sentinel = utils.non_linear_projection(tf.concat([que_enc, sentinel_q], axis = 1))


# ===================  COATTENTION ENCODER ===================
# L \in R(doc_size + 1) x (que_size + 1)
L = tf.matmul(doc_enc_sentinel, que_enc_sentinel, transpose_a = True)
A_Q = tf.nn.softmax(L, 0)
A_D = tf.nn.softmax(tf.transpose(L), 1)
C_Q = tf.matmul(doc_enc_sentinel, A_Q)
# C_D \in R_2*lstm_size x (doc_size + 1)
C_D = tf.matmul(tf.concat([que_enc_sentinel, C_Q], axis = 0), A_D)


bi_lstm_input = tf.unstack(tf.reshape(tf.transpose(tf.concat([doc_enc_sentinel, C_D], axis = 0)), [max_sequence_length + 1, 1, 3*lstm_size]))
outputs_bi, output_state_fw, output_state_bw = tf.nn.static_bidirectional_rnn(lstm_cenc_fw, lstm_cenc_bw, inputs = bi_lstm_input , dtype=tf.float32)
U = tf.transpose(tf.squeeze(tf.slice(outputs_bi, [0,0,0], [document_size, 1,  2*lstm_size])))
#U.set_shape([2*lstm_size, document_size])
print(U)


Tensor("transpose_6:0", dtype=float32)


In [9]:
# ===================== DYNAMIC POINTING DECODER =============


#scope = tf.get_variable_scope()
#u_t = get_scope_variable(scope, 'hmn_u_t', [2*lstm_size, 1]) 
#h_i = get_scope_variable(scope, 'hmn_h_i', [lstm_size, 1 ]) 
#u_s_i = get_scope_variable(scope, 'hmn_u_s_i', [2*lstm_size, 1])
#u_e_i = get_scope_variable(scope, 'hmn_u_e_i', [2*lstm_size, 1])


#m_3 = HMN(U, h_i, u_s_i, u_e_i)
#print(m_3)

# returns tuple (scores_start, scores_end, strart_pos, start_end, new_lstm_state)
def decoderIteration(U, lstm_state, start_pos, end_pos):
    #print(tf.slice(U, [0,0], [lstm_size*2, 0]))
    #print(tf.slice(U, [0, start_pos], [lstm_size*2, 1], name='slice-1x'))
    
    scores_start = hmn.HMN(U, 
                       tf.transpose(lstm_state.h), 
                       tf.slice(U, [0, start_pos], [lstm_size*2, 1]) ,
                       tf.slice(U, [0, end_pos], [lstm_size*2, 1],) , 
                       'start',
                        FLAGS)

    new_start_pos = tf.to_int32(tf.argmax(scores_start, 0))

    #print(lstm_state)
    scores_end = hmn.HMN(U, 
                     tf.transpose(lstm_state.h), 
                     tf.slice(U, [0, new_start_pos], [lstm_size*2, 1],) ,
                     tf.slice(U, [0, end_pos], [lstm_size*2, 1],), 
                    'end',
                    FLAGS)
    new_end_pos = tf.to_int32(tf.argmax(scores_end, 0))
    lstm_input = tf.concat(
        [tf.slice(U, [0, new_start_pos], [lstm_size*2, 1], name='slice-5'), tf.slice(U, [0, new_end_pos], [lstm_size*2, 1])],
        axis = 0
    )
    output, new_lstm_state = lstm_dec(tf.reshape(lstm_input, [1, lstm_size*4]), lstm_state)
    #print(new_lstm_state)
    return scores_start, scores_end, new_start_pos , new_end_pos, new_lstm_state


start_pos = 0;
end_pos = 0;
sum_start_scores = tf.zeros([1, document_size])
sum_end_scores = tf.zeros([1, document_size])
lstm_dec_state = lstm_dec.zero_state(1, tf.float32)
#print(lstm_dec_state)

for step in range(max_decoder_iterations):
    scores_start, scores_end, new_start_pos, new_end_pos, lstm_dec_state = decoderIteration(U, lstm_dec_state, start_pos, end_pos)
    sum_start_scores = tf.add(sum_start_scores, scores_start)
    sum_end_scores   = tf.add(sum_end_scores, scores_end)
    if new_start_pos == start_pos and end_pos == new_end_pos : break
    start_pos = new_start_pos
    end_pos = new_end_pos

    
# loss and train step
start_end_true = tf.placeholder(tf.int32, [2]);
#end_true = tf.placeholder(tf.int32, ());
onehot_labels = tf.one_hot(start_end_true, document_size)
sum_loss = tf.losses.softmax_cross_entropy(
    onehot_labels,
    tf.concat([sum_start_scores, sum_end_scores], axis=0))

#print(sum_loss)


train_step = optimizer.minimize(sum_loss)


In [13]:
#=========== Training ==================
dataset = ds.getDataset(["./train_train_task_b.csv"], max_sequence_length)
iterator = dataset.make_one_shot_iterator()
next_element = iterator.get_next()

init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch_ in range(max_epoch):
        for step_ in range(5):
            n = sess.run(next_element)
            print(n[0])
            print(n[1])
            
            sess.run(train_step, feed_dict={question_ph: [n[5]], document_ph: [n[4]], start_end_true: [n[0], n[1]]})
            print(sess.run(sum_loss, feed_dict={question_ph: [n[5]], document_ph: [n[4]], start_end_true: [n[0], n[1]]}))
    print('End')


24
24
0.0
43
45
0.0
107
110
0.0
err i
err u
106
108
0.0
73
75
0.0
End


In [42]:
print(tf.nn.softmax(tf.convert_to_tensor([1,2], dtype=tf.float32)))

Tensor("Reshape_550:0", shape=(2,), dtype=float32)


In [19]:
def ff(a):
    def _ff(b):
        return a + b
    return _ff

In [20]:
a = ff(4)

In [23]:
print(a)

<function ff.<locals>._ff at 0x7fee89f202f0>
